In [1]:
!pip install rdflib pyvis

In [2]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import RDF, FOAF

# Create a new RDFLib graph
g = Graph()

# Define namespaces
EX = Namespace("http://example.org/")
SCHEMA = Namespace("http://schema.org/")

# Add namespaces to the graph
g.bind("ex", EX)
g.bind("schema", SCHEMA)
g.bind("foaf", FOAF)

In [3]:
# Function to add a new book entry to the graph
def add_book_entry(graph, isbn, title, author_name):
    author_uri = URIRef(EX[author_name.replace(' ', '_')]) # Author url of J. K. Rowling is J._K._Rowling
    print(author_uri)
    
    if not (author_uri, RDF.type, FOAF.Person) in graph:
        graph.add((author_uri, RDF.type, FOAF.Person))
        graph.add((author_uri, FOAF.name, Literal(author_name)))
    
    book_uri = URIRef(EX[isbn]) # Book url is the isbn
    
    graph.add((book_uri, RDF.type, SCHEMA.Book)) # Add book type
    graph.add((book_uri, SCHEMA.name, Literal(title))) # Add name to book
    graph.add((book_uri, SCHEMA.author, author_uri)) # Connect author to book
    graph.add((book_uri, SCHEMA.identifier, Literal(isbn))) # Add isbn as identifier

In [4]:
# Make sure key is one of RDF... or SCHEMA...
def add_property_to_book(isbn, key, value):
    book_uri = URIRef(EX[isbn]) # Book url is the isbn
    
    if not (book_uri, RDF.type, SCHEMA.Book) in graph:
        raise ValueError('This isbn does not exist in graph')
        
    graph.add((book_uri, key, value))

In [ ]:
# Make sure key is one of RDF... or SCHEMA...
def add_property_to_author(author, key, value):
    author_uri = author
    if type(author) == str:
        author_uri = URIRef(EX[author_name.replace(' ', '_')])
    
    if not (author_uri, RDF.type, FOAF.Person) in graph:
        raise ValueError('This author does not exist in graph')
        
    graph.add((author_uri, key, value))

In [5]:
add_book_entry(g, "978-0439139595", "Harry Potter and the Goblet of Fire", "J.K. Rowling")

http://example.org/J.K._Rowling


In [6]:
from pyvis.network import Network
import webbrowser

# Jupyter notebook only
def visualize_rdf_graph(graph):
    net = Network(notebook=True)
    
    author_color = 'red'
    book_color = 'green'
    
    for s, p, o in graph:
        s_str = str(s)
        o_str = str(o)
        
        # Determine the color based on the type of node
        
        # For subject
        if (s, RDF.type, FOAF.Person) in graph:
            net.add_node(s_str, label=s_str, title=s_str, color=author_color)
        elif (s, RDF.type, SCHEMA.Book) in graph:
            net.add_node(s_str, label=s_str, title=s_str, color=book_color)
        else:
            net.add_node(s_str, label=s_str, title=s_str)
        
        # For object
        if (o, RDF.type, FOAF.Person) in graph:
            net.add_node(o_str, label=o_str, title=o_str, color=author_color)
        elif (o, RDF.type, SCHEMA.Book) in graph:
            net.add_node(o_str, label=o_str, title=o_str, color=book_color)
        else:
            net.add_node(o_str, label=o_str, title=o_str)
        
        net.add_edge(s_str, o_str, title=str(p))
    
    # Show the network
    net.show("rdf_graph.html")
    
    # And open it in a new tab (again, Jupyter Notebook only)
    webbrowser.open('rdf_graph.html')

# Visualize the RDF graph
visualize_rdf_graph(g)

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


In [7]:
# Export for use in GraphDB
with open("books.rdf", "w") as f:
    f.write(g.serialize(format="xml"))